In [59]:
import pandas as pd
from pandas.api.types import is_float_dtype, is_integer_dtype, is_object_dtype
import os
from dotenv import load_dotenv
from supabase import create_client, Client
import psycopg2
from psycopg2 import sql
import unicodedata
from google import genai
from google.genai import types

In [60]:
load_dotenv()

file_name = "Deliberation_2024_2025_L2_S3"
url: str = os.getenv("SUPABASE_URL")
key: str = os.getenv("SUPABASE_KEY")
key_llm = os.getenv("GOOGLE_API_KEY")
db_url: str = os.getenv("DATABASE_URL")
client: Client = create_client(url, key)
client_llm = genai.Client(api_key=key_llm)

In [61]:
data = pd.read_csv(f"../data/{file_name}_postprocess.csv")

def replace_french_e(text):
    normalized_text = unicodedata.normalize('NFD', text)
    transformed_text = ''.join(char for char in normalized_text if not unicodedata.combining(char))
    return transformed_text

def remove_newlines(text_list):
    for i in range(len(text_list)):
        text_list[i] = text_list[i].replace('\n', '')
    return text_list

In [62]:
sys_instruct = """
You are a text formatting assistant specializing in data preparation for databases. When given an array of elements, follow these precise instructions:  

- Accept any array of string elements.  
- Convert all characters to lowercase.  
- Replace all spaces and special characters (e.g., apostrophes, parentheses) with underscores (`_`).  
- Remove any characters that could make the column name invalid in a database (e.g., starting with a number, containing non-alphanumeric characters except underscores, or exceeding typical column name length limits).  
- Ensure column names follow standard database conventions for improved compatibility.  
- **Ensure the final output is a single, strictly comma-separated string with no spaces between elements and no trailing comma.**  

**Example Input:**  
Name,Code,Compilation,Système d'exploitation 2,Moyenne UE 1,Crédit UE 1,Génie Logiciel 2,Interface Machine (Homme),Moyenne UE 2,Crédit UE 2,Probabilités et Statistiques,Programmation (Linéaire),Moyenne UE 3,Crédit UE 3,Economie et veille stratégique numérique,Moyenne UE 4,Crédit UE 4,Crédits du Semestre,Moyenne du Semestre  

**Expected Output:**  
`name,code,compilation,systeme_d_exploitation_2,moyenne_ue_1,credit_ue_1,genie_logiciel_2,interface_machine_homme,moyenne_ue_2,credit_ue_2,probabilites_et_statistiques,programmation_lineaire,moyenne_ue_3,credit_ue_3,economie_et_veille_strategique_numerique,moyenne_ue_4,credit_ue_4,credits_du_semestre,moyenne_du_semestre`  

Respond only with the formatted output unless otherwise instructed.
"""

def generate_db_names(names):
    response = client_llm.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=sys_instruct),
    contents=[names],
    )
    return response.text

In [ ]:
# creating mappings & db column names
db_cols = generate_db_names(",".join(data.columns.to_list())).split(",")
db_cols = remove_newlines(db_cols)
cols = data.columns.to_list()
mappings = dict(zip(cols, db_cols))
print(mappings)

{'Name': 'name', 'Code': 'code', 'Systèmes d’information': 'systemes_d_information', 'Théorie des graphes': 'theorie_des_graphes', 'Moyenne UE 1': 'moyenne_ue_1', 'Crédit UE 1': 'credit_ue_1', 'Algorithmique et Structures de Données (ASD)': 'algorithmique_et_structures_de_donnees_asd', 'Architecture des Ordinateurs (AO)': 'architecture_des_ordinateurs_ao', 'Moyenne UE 2': 'moyenne_ue_2', 'Crédit UE 2': 'credit_ue_2', 'Mathématique Logique': 'mathematique_logique', 'Méthodes Numériques': 'methodes_numeriques', 'Moyenne UE 3': 'moyenne_ue_3', 'Crédit UE 3': 'credit_ue_3', 'Langue Étrangère 2': 'langue_etrangere_2', 'Moyenne UE 4': 'moyenne_ue_4', 'Crédit UE 4': 'credit_ue_4', 'Crédits du Semestre': 'credits_du_semestre', 'Moyenne du Semestre': 'moyenne_du_semestre'}


In [ ]:
schema = []
table_cols = data.columns.to_list()

for col in table_cols:
    is_int = is_integer_dtype(data[col])
    is_float = is_float_dtype(data[col])
    is_object = is_object_dtype(data[col])

    if is_float:
        schema.append([mappings[col], "float8"])
    if is_int:
        schema.append([mappings[col], "int8"])
    if is_object:
        schema.append([mappings[col], "text"])

dict_schema = dict(schema)

# table name 
table_name = f"analysis_{file_name}"
columns = ", ".join([f"{col} {dtype}" for col, dtype in dict_schema.items()])

# table creation query
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    id UUID DEFAULT gen_random_uuid() PRIMARY KEY,
    {columns},
    UNIQUE(code)
)
"""

# transforming col names
data.columns = [mappings[col] for col in data.columns]

data.head()

,name,code,systemes_d_information,theorie_des_graphes,moyenne_ue_1,credit_ue_1,algorithmique_et_structures_de_donnees_asd,architecture_des_ordinateurs_ao,moyenne_ue_2,credit_ue_2,mathematique_logique,methodes_numeriques,moyenne_ue_3,credit_ue_3,langue_etrangere_2,moyenne_ue_4,credit_ue_4,credits_du_semestre,moyenne_du_semestre
0,ABDAOUI AYA,212136023497,13.30,6.20,10.46,9,6.55,8.42,7.49,0,10.10,8.10,9.10,4,10.0,10.0,2,15,8.98
1,ABDESMED CHOUROUK,232336177720,13.90,8.75,11.84,9,4.38,8.50,6.44,0,11.95,11.40,11.68,8,14.0,14.0,2,19,9.91
2,ABDI SAMY ZAKARIA,222236128502,0.00,0.00,0.00,0,0.00,0.00,0.00,0,0.00,0.00,0.00,0,0.0,0.0,0,0,0.00
3,ABDOUN CELIA NOOR,232336124105,12.20,4.07,8.95,5,3.34,7.50,5.42,0,6.60,4.20,5.40,0,16.0,16.0,2,7,7.18
4,ABIDI LYNA,232336204903,14.15,6.60,11.13,9,3.45,8.10,5.78,0,12.00,8.12,10.06,8,14.0,14.0,2,19,9.04


In [65]:
conn_details = {
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT"),
    "sslmode": os.getenv("DB_SSLMODE")
}



try:
    conn = psycopg2.connect(**conn_details)
    print("Connection successful!")

    cursor = conn.cursor()
    cursor.execute(
        sql.SQL("""
            SELECT EXISTS (
                SELECT 1 FROM information_schema.tables
                WHERE table_name = %s
            );
        """),
        [table_name.lower()]
    )

    table_exists = cursor.fetchone()[0]

    if table_exists:
        print(f"Table '{table_name}' already exists.")
    else:
        cursor.execute(create_table_query)
        conn.commit()
        print(f"Table '{table_name}' created successfully.")

    cursor.close()
    conn.close()

except Exception as e:
    print(f"Failed to connect or execute query: {e}")


Connection successful!
Table 'analysis_Deliberation_2024_2025_L2_S3' created successfully.


In [66]:
i = 0
records = data.to_dict(orient="records")

try:
    response = client.table(table_name.lower()).upsert(records).execute()
    print("Records inserted successfully")
except Exception as e:
    print(f"Error inserting records: {e.message}")

Records inserted successfully


In [67]:
# create mappings table
mappings_table_name = f"{table_name}_mappings"
mappings_data = [{"name": name, "db_name": mappings[name]} for name in mappings]
mappings_schema = {"name": "text", "db_name": "text"}
mappings_cols = "name text, db_name text"
create_mappings_table_query = f"""
CREATE TABLE IF NOT EXISTS {mappings_table_name} (
    id UUID DEFAULT gen_random_uuid() PRIMARY KEY,
    {mappings_cols},
    UNIQUE(db_name)
)
"""

try:
    conn = psycopg2.connect(**conn_details)
    print("Connection successful!")

    cursor = conn.cursor()
    cursor.execute(
        sql.SQL("""
            SELECT EXISTS (
                SELECT 1 FROM information_schema.tables
                WHERE table_name = %s
            );
        """),
        [mappings_table_name.lower()]
    )

    table_exists = cursor.fetchone()[0]

    if table_exists:
        print(f"Table '{mappings_table_name}' already exists.")
    else:
        cursor.execute(create_mappings_table_query)
        conn.commit()
        print(f"Table '{mappings_table_name}' created successfully.")

    cursor.close()
    conn.close()

except Exception as e:
    print(f"Failed to connect or execute query: {e}")

response = client.table(mappings_table_name.lower()).upsert(mappings_data).execute()
print("Mappings inserted successfully")

Connection successful!
Table 'analysis_Deliberation_2024_2025_L2_S3_mappings' created successfully.
Mappings inserted successfully
